# This notebook is meant to research denoising techniques for the already cropped receipt images.

The receipt images show various types of noise, such as:
- Shadows
- Wrinkles
- Blurry text
- etc.

Perhaps we can apply some denoising techniques to improve OCR results that are to come.

There are many denoising techniques available. We will of course not test them all, but we can try a few and see if they help.

We could try some "physical" denoising techniques (filtering, morphological operations, etc.), some statistical denoising techniques (like PCA-based denoising) as well as some "deep learning" denoising techniques (like Denoising Autoencoders).

In [7]:
import os, sys
import numpy as np
import cv2
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

sys.path.append(os.path.abspath(os.path.join('../data')))
sys.path.append(os.path.abspath(os.path.join('..')))

---

## Assessing the effect of blurred components in the images

In [31]:
# Loading an image

cropped_images_dir = "../data/_debug/"
image_filenames = os.listdir(cropped_images_dir)

print(f"Number of cropped images available for denoising exmperiments: {len(image_filenames)}")

Number of cropped images available for denoising exmperiments: 66
